In [1]:
import os
import logging
import pandas as pd
import mysql.connector

In [11]:
MYSQL_USERNAME=os.environ['MYSQL_USERNAME']
MYSQL_PASSWORD=os.environ['MYSQL_PASSWORD']
MYSQL_DATABASE=os.environ['MYSQL_DATABASE']
MYSQL_HOST=os.environ['MYSQL_HOST']

try:
    # Establish connection
    conn = mysql.connector.connect(
        host=MYSQL_HOST,  # Change to your MySQL server
        user=MYSQL_USERNAME,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE  # Optional: specify database
    )

    if conn.is_connected():
        print("Connected to MySQL successfully!")

except mysql.connector.Error as e:
    print("Error:", e)

Connected to MySQL successfully!


In [12]:
# Create a cursor object
cursor = conn.cursor()

In [13]:
def extract(cursor, query):
    # Execute a query
    cursor.execute(query)
    data = cursor.fetchall()
    
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    return df

In [14]:
query="SELECT * FROM Customer;"
customers_df = extract(cursor, query)

In [15]:
customers_df

,CustomerID,CustomerName,Phone,Gender,Age,City
0,1,Alice Wilson,555-0001,Female,34,New York
1,2,David Lee,555-0002,Male,28,Los Angeles
2,3,Sarah Gonzalez,555-0003,Female,45,Chicago
3,4,Brian Kim,555-0004,Male,22,Houston
4,5,Nina Patel,555-0005,Female,30,San Francisco
5,6,Tom Nguyen,555-0006,Male,40,Miami
6,7,Jessica Chen,555-0007,Female,29,Seattle
7,8,Ryan Adams,555-0008,Male,33,Dallas
8,9,Emma Robinson,555-0009,Female,26,Austin
9,10,Liam Thompson,555-0010,Male,31,Boston


In [16]:
# Close the connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.


In [17]:
import psycopg2
from psycopg2 import sql

In [18]:
PG_USERNAME=os.environ['PG_USERNAME']
PG_PASSWORD=os.environ['PG_PASSWORD']
PG_DATABASE=os.environ['PG_DATABASE']
PG_HOST=os.environ['PG_HOST']

print(PG_HOST, PG_USERNAME, PG_PASSWORD, PG_DATABASE)

172.18.0.2 biruni P@ssw0rd walmart_dwh


In [19]:
# Database connection parameters
db_config = {
    'dbname': PG_DATABASE,
    'user': PG_USERNAME,
    'password': PG_PASSWORD,
    'host': PG_HOST,  # or your database host
    'port': 5432          # default PostgreSQL port
}

In [20]:
# Connect to the database
try:
    conn = psycopg2.connect(**db_config)
    print("Connected to the database!")
except Exception as e:
    print(f"Unable to connect to the database: {e}")


Connected to the database!


In [21]:
# Create a cursor object to interact with the database
cursor = conn.cursor()

In [25]:
query = sql.SQL(f"""
    INSERT INTO dimcustomer (customerbk, customername, phone, gender, age, city )
    VALUES (%s, %s, %s, %s, %s, %s);
""")

In [26]:
def load(cursor, query, data):
    for i, row in data.iterrows():

        try:
            cursor.execute(query, tuple(row))
        except Exception as e:
            conn.rollback()  # Rollback in case of error
            print(f"Error loading cutomer: {e}")

    cursor.execute('commit')
    print("Data Loaded successfully!")

In [27]:
load(cursor, query, customers_df)

Data Loaded successfully!


In [28]:
# Close the cursor and connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.
